In [1]:
!pip install transformers
!pip install datasets
!pip install seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 45.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 60.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 16.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 KB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for

In [49]:
from datasets import load_dataset, load_metric
import transformers
import tensorflow as tf

In [4]:
task = "ner"  # Should be one of "ner", "pos" or "chunk"
model_checkpoint = "distilbert-base-uncased"
batch_size = 32

#1. Loading the dataset

In [5]:
datasets = load_dataset('conll2003')

Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

Dataset conll2003 downloaded and prepared to /root/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [15]:
datasets['train'][2]

{'id': '2',
 'tokens': ['BRUSSELS', '1996-08-22'],
 'pos_tags': [22, 11],
 'chunk_tags': [11, 12],
 'ner_tags': [5, 0]}

In [8]:
datasets['train'].features['ner_tags']

Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)

So for the NER tags, 0 corresponds to 'O', 1 to 'B-PER' etc... On top of the 'O' (which means no special entity), there are four labels for NER here, each prefixed with 'B-' (for beginning) or 'I-' (for intermediate), that indicate if the token is the first one for the current group with the label or not:
- 'PER' for person
- 'ORG' for organization
- 'LOC' for location
- 'MISC' for miscellaneous

In [17]:
label_list = datasets['train'].features[f'{task}_tags'].feature.names
label_list

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [51]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML


def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(
        dataset
    ), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset) - 1)
        while pick in picks:
            pick = random.randint(0, len(dataset) - 1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(
                lambda x: [typ.feature.names[i] for i in x]
            )
    display(HTML(df.to_html()))
show_random_elements(datasets["train"])

,id,tokens,pos_tags,chunk_tags,ner_tags
0,282,"[The, wells, will, be, reperforated, and, gravel, will, be, pumped, into, the, reservoir, through, one, of, the, wells, to, avoid, plugging, problems, in, the, future, ,, it, said, .]","[DT, NNS, MD, VB, VBN, CC, NN, MD, VB, VBN, IN, DT, NN, IN, CD, IN, DT, NNS, TO, VB, VBG, NNS, IN, DT, NN, ,, PRP, VBD, .]","[B-NP, I-NP, B-VP, I-VP, I-VP, O, B-NP, B-VP, I-VP, I-VP, B-PP, B-NP, I-NP, B-PP, B-NP, B-PP, B-NP, I-NP, B-VP, I-VP, I-VP, B-NP, B-PP, B-NP, I-NP, O, B-NP, B-VP, O]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
1,9916,"[The, agency, said, it, would, monitor, effects, of, the, two, new, rules, before, considering, the, "", price, improvement, "", proposal, again, .]","[DT, NN, VBD, PRP, MD, VB, NNS, IN, DT, CD, JJ, NNS, IN, VBG, DT, "", NN, NN, "", NN, RB, .]","[B-NP, I-NP, B-VP, B-NP, B-VP, I-VP, B-NP, B-PP, B-NP, I-NP, I-NP, I-NP, B-PP, B-VP, B-NP, O, B-NP, I-NP, O, B-NP, B-ADVP, O]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
2,3628,"[Stranraer, 1, Livingston, 2]","[NNP, CD, NNP, CD]","[B-NP, I-NP, I-NP, I-NP]","[B-ORG, O, B-ORG, O]"
3,5721,"[Halftime, 1-1, .]","[VB, NNP, .]","[B-VP, B-NP, O]","[O, O, O]"
4,517,"[Scorers, :, Lubarskij, (, 50th, minute, ), ,, Valkucak, (, 54th, )]","[NNS, :, NNP, (, JJ, NN, ), ,, NNP, (, CD, )]","[B-NP, O, B-NP, O, B-NP, I-NP, O, O, B-NP, O, B-NP, O]","[O, O, B-PER, O, O, O, O, O, B-PER, O, O, O]"
5,11449,"[--, Jerry, Bieszk, 312-408-8725]","[:, NNP, NNP, CD]","[B-NP, I-NP, I-NP, I-NP]","[O, B-PER, I-PER, O]"
6,2102,"[DETROIT, AT, KANSAS, CITY]","[NNP, NNP, VBD, NNP]","[B-NP, I-NP, B-VP, B-NP]","[B-ORG, O, B-LOC, I-LOC]"
7,13311,"[First, Alliance, Corporation, and, Subsidiaries]","[NNP, NNP, NNP, CC, NNP]","[B-NP, I-NP, I-NP, I-NP, I-NP]","[B-ORG, I-ORG, I-ORG, I-ORG, I-ORG]"
8,523,"[Gloria, Bistrita, -, Ilie, Lazar, (, 32nd, ), ,, Eugen, Voica, (, 84th, )]","[NNP, NNP, :, NNP, NNP, (, CD, ), ,, NNP, NNP, (, CD, )]","[B-NP, I-NP, O, B-NP, I-NP, O, B-NP, O, O, B-NP, I-NP, O, B-NP, O]","[B-ORG, I-ORG, O, B-PER, I-PER, O, O, O, O, B-PER, I-PER, O, O, O]"
9,11677,"[PRECIOUS, METALS]","[JJ, NNS]","[B-NP, I-NP]","[O, O]"


# 2. Preprocessing the data

In [18]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [21]:
tokenizer("Hello, this is a test sentence")

{'input_ids': [101, 7592, 1010, 2023, 2003, 1037, 3231, 6251, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [25]:
example = datasets['train'][4]
print(example['tokens'])

['Germany', "'s", 'representative', 'to', 'the', 'European', 'Union', "'s", 'veterinary', 'committee', 'Werner', 'Zwingmann', 'said', 'on', 'Wednesday', 'consumers', 'should', 'buy', 'sheepmeat', 'from', 'countries', 'other', 'than', 'Britain', 'until', 'the', 'scientific', 'advice', 'was', 'clearer', '.']


In [26]:
tokenized_input = tokenizer(example['tokens'], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input['input_ids'])
print(tokens)

['[CLS]', 'germany', "'", 's', 'representative', 'to', 'the', 'european', 'union', "'", 's', 'veterinary', 'committee', 'werner', 'z', '##wing', '##mann', 'said', 'on', 'wednesday', 'consumers', 'should', 'buy', 'sheep', '##me', '##at', 'from', 'countries', 'other', 'than', 'britain', 'until', 'the', 'scientific', 'advice', 'was', 'clearer', '.', '[SEP]']


In [28]:
print(tokenized_input.word_ids())

[None, 0, 1, 1, 2, 3, 4, 5, 6, 7, 7, 8, 9, 10, 11, 11, 11, 12, 13, 14, 15, 16, 17, 18, 18, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, None]


In [27]:
len(example[f"{task}_tags"]), len(tokenized_input["input_ids"])

(31, 39)

In [33]:
# quy định chiến lược đặt tất cả token đặc biệt thành -100, các token phụ sẽ được đặt cùng một nhãn với token chính
label_all_tokens = True

In [34]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [42]:
tokenize_and_align_labels(datasets["train"][:5])

{'input_ids': [[101, 7327, 19164, 2446, 2655, 2000, 17757, 2329, 12559, 1012, 102], [101, 2848, 13934, 102], [101, 9371, 2727, 1011, 5511, 1011, 2570, 102], [101, 1996, 2647, 3222, 2056, 2006, 9432, 2009, 18335, 2007, 2446, 6040, 2000, 10390, 2000, 18454, 2078, 2329, 12559, 2127, 6529, 5646, 3251, 5506, 11190, 4295, 2064, 2022, 11860, 2000, 8351, 1012, 102], [101, 2762, 1005, 1055, 4387, 2000, 1996, 2647, 2586, 1005, 1055, 15651, 2837, 14121, 1062, 9328, 5804, 2056, 2006, 9317, 10390, 2323, 4965, 8351, 4168, 4017, 2013, 3032, 2060, 2084, 3725, 2127, 1996, 4045, 6040, 2001, 24509, 1012, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, -100], [-100, 1, 2, -100], [-100, 5, 0, 

In [43]:
# Áp dụng cho dataset
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [45]:
tokenized_datasets['train']['labels'][0]

[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, -100]

# 3. Fine-tuning the model

In [47]:
from transformers import TFAutoModelForTokenClassification

id2label = {i: label for i, label in enumerate(label_list)}
label2id = {label: i for i, label in enumerate(label_list)}

model = TFAutoModelForTokenClassification.from_pretrained(
    model_checkpoint, num_labels=len(label_list), id2label=id2label, label2id=label2id
)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForTokenClassification: ['activation_13', 'vocab_layer_norm', 'vocab_projector', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier', 'dropout_19']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inferenc

In [50]:
from transformers import create_optimizer

num_train_epochs = 3
num_train_steps = (len(tokenized_datasets['train']) // batch_size) * num_train_epochs
optimizer, lr_schedule = create_optimizer(
    init_lr=3e-5,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
    num_warmup_steps=0
)

model.compile(optimizer=optimizer)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [53]:
from transformers import DataCollatorForTokenClassification

# DataCollatorForTokenClassification sẽ gộp các ví dụ đã xử lý của chúng ta lại với nhau trong khi áp dụng phần đệm để làm cho chúng có cùng kích thước 
data_collator = DataCollatorForTokenClassification(tokenizer, return_tensors="np")

In [54]:
train_ds = model.prepare_tf_dataset(
    tokenized_datasets['train'],
    shuffle=True,
    batch_size=batch_size,
    collate_fn=data_collator
)

val_ds = model.prepare_tf_dataset(
    tokenized_datasets['validation'],
    shuffle=False,
    batch_size=batch_size,
    collate_fn=data_collator
)

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [60]:
import numpy as np
from transformers.keras_callbacks import KerasMetricCallback

metric = load_metric('seqeval')
labels = [label_list[i] for i in example[f'{task}_tags']]
metric.compute(predictions=[labels], references=[labels])


{'LOC': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'ORG': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'PER': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

In [61]:
def compute_matrics(p):
  predictions, labels = p
  predictions =   np.argmax(predictions, axis=2)

  # Xóa token đặc biệt
  true_predictions = [
      [label_list[p] for (p,l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)
  ]
  true_labels = [
      [label_list[l] for (p,l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)
  ]

  results = metric.compute(predictions=true_predictions, references=true_labels)
  return {
      'precision': results['overall_precision'],
      'recall': results['overall_recall'],
      'f1': results['overall_f1'],
      'accuracy': results['overall_accuracy']
  }

metric_callback = KerasMetricCallback(
    metric_fn=compute_matrics, eval_dataset=val_ds
)

In [63]:
from tensorflow.keras.callbacks import TensorBoard

tensorboard_callback = TensorBoard(log_dir="./tc_model_save/logs")

callbacks = [metric_callback, tensorboard_callback]

history = model.fit(train_ds,
                    validation_data=val_ds,
                    epochs=num_train_epochs,
                    callbacks=callbacks)

Epoch 1/3


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:717: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)


  6/438 [..............................] - ETA: 1:15 - loss: 1.8295

438/438 [==============================] - 119s 244ms/step - loss: 0.2192 - val_loss: 0.0689 - precision: 0.8983 - recall: 0.9165 - f1: 0.9073 - accuracy: 0.9793
Epoch 2/3
438/438 [==============================] - 104s 237ms/step - loss: 0.0538 - val_loss: 0.0581 - precision: 0.9219 - recall: 0.9327 - f1: 0.9273 - accuracy: 0.9829
Epoch 3/3
438/438 [==============================] - 104s 238ms/step - loss: 0.0336 - val_loss: 0.0569 - precision: 0.9227 - recall: 0.9365 - f1: 0.9295 - accuracy: 0.9836


In [64]:
!tensorboard dev upload --logdir ./tc_model_save/logs \
  --name "NER with transformers" \
  --description "Training results for NER with fine-tuning distilbert-base-uncased" \
  --one_shot

2023-03-08 10:34:01.586200: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-08 10:34:01.586323: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-08 10:34:01.586345: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.

***** TensorBoard Uploader *****

This will upload your TensorBoard logs to https://tensorboard.dev/ from
the following directory:

./tc_model_save/logs

This TensorB

In [67]:
import pickle 
save_path = '/content/drive/MyDrive/ner_with_transformers/'
# Lưu model
model.save_pretrained(save_path+'model')
# Lưu thông tin của model
with open(save_path+'info.pkl', 'wb') as f:
    pickle.dump((model_checkpoint), f)

# 4. Inference

In [83]:
sample_sentence = "Vietnam, country occupying the eastern portion of mainland Southeast Asia."
tokenized = tokenizer([sample_sentence], return_tensors="np")

In [84]:
outputs = model(tokenized).logits
classes = np.argmax(outputs, axis=-1)[0]
print(classes)

[0 5 0 0 0 0 0 0 0 0 5 6 0 5]


In [85]:
outputs = [(tokenizer.decode(token), model.config.id2label[id]) for token, id in zip(tokenized["input_ids"][0], classes)]
print(outputs)

[('[CLS]', 'O'), ('vietnam', 'B-LOC'), (',', 'O'), ('country', 'O'), ('occupying', 'O'), ('the', 'O'), ('eastern', 'O'), ('portion', 'O'), ('of', 'O'), ('mainland', 'O'), ('southeast', 'B-LOC'), ('asia', 'I-LOC'), ('.', 'O'), ('[SEP]', 'B-LOC')]
